In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

X=pd.read_csv('train.csv',index_col='Id')
X_test=pd.read_csv('test.csv',index_col='Id')


In [2]:
X.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [5]:
X.dropna(axis=0,subset=['SalePrice'],inplace=True)
y=X.SalePrice
y

Id
1       208500
2       181500
3       223500
4       140000
5       250000
         ...  
1456    175000
1457    210000
1458    266500
1459    142125
1460    147500
Name: SalePrice, Length: 1460, dtype: int64

In [6]:
X=X.select_dtypes(exclude=['object'])
X_test=X_test.select_dtypes(exclude=['object'])

In [7]:
X_train,X_valid,y_train,y_valid=train_test_split(X,y,train_size=.8,test_size=0.2,random_state=0)

In [8]:
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
619,20,90.0,11694,9,5,2007,2007,452.0,48,0,...,0,108,0,0,260,0,0,7,2007,314813
871,20,60.0,6600,5,5,1962,1962,0.0,0,0,...,0,0,0,0,0,0,0,8,2009,109500
93,30,80.0,13360,5,7,1921,2006,0.0,713,0,...,0,0,44,0,0,0,0,8,2009,163500
818,20,NaN,13265,8,5,2002,2002,148.0,1218,0,...,150,59,0,0,0,0,0,7,2008,271000
303,20,118.0,13704,7,5,2001,2002,150.0,0,0,...,468,81,0,0,0,0,0,1,2006,205000


In [9]:
X_train.isnull().sum()

MSSubClass         0
LotFrontage      212
LotArea            0
OverallQual        0
OverallCond        0
YearBuilt          0
YearRemodAdd       0
MasVnrArea         6
BsmtFinSF1         0
BsmtFinSF2         0
BsmtUnfSF          0
TotalBsmtSF        0
1stFlrSF           0
2ndFlrSF           0
LowQualFinSF       0
GrLivArea          0
BsmtFullBath       0
BsmtHalfBath       0
FullBath           0
HalfBath           0
BedroomAbvGr       0
KitchenAbvGr       0
TotRmsAbvGrd       0
Fireplaces         0
GarageYrBlt       58
GarageCars         0
GarageArea         0
WoodDeckSF         0
OpenPorchSF        0
EnclosedPorch      0
3SsnPorch          0
ScreenPorch        0
PoolArea           0
MiscVal            0
MoSold             0
YrSold             0
SalePrice          0
dtype: int64

In [10]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

In [11]:
from sklearn.impute import SimpleImputer


my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))


imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns


In [12]:
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

887.4611643835614


In [13]:
final_imputer = SimpleImputer(strategy='median')
final_X_train = pd.DataFrame(final_imputer.fit_transform(X_train))
final_X_valid = pd.DataFrame(final_imputer.transform(X_valid))


final_X_train.columns = X_train.columns
final_X_valid.columns = X_valid.columns


In [15]:
model = RandomForestRegressor(n_estimators=100, random_state=0)
model.fit(final_X_train, y_train)

preds_valid = model.predict(final_X_valid)
print("MAE (Mean approach):")
print(mean_absolute_error(y_valid, preds_valid))

MAE (Mean approach):
919.0409931506856
